In [62]:
import random
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import (mean_absolute_error, mean_squared_error)

In [63]:
df = pd.read_excel("NFL Champion RawData.xlsx")
df.head(2)

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,sum all pro,sum cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2023,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.86,0.0,0.78,9,0.0,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2023,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.00,0.0,0.00,22,0.0,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73


In [64]:
#Dropping all data before 2001, as this dataset begins in 1991, so stats don't fully come into focus until 2001s.
#All data cleaning was done prior in excel

#df.drop(df[df['Season'] < 1991].index, inplace = True)
df

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,sum all pro,sum cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2023,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.86,0.00,0.78,9,0.00,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2023,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.00,0.00,0.00,22,0.00,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73
2,2023,19,19 2022,San Francisco 49ers,San Francisco 49ers 2022,0.00,0.00,0.24,10,0.30,...,1321,11,3.4,77,96,832,26,25.7,15.3,56.26
3,2023,2,2 2022,Buffalo Bills,Buffalo Bills 2022,0.08,0.38,0.06,16,0.16,...,1673,10,4.3,103,89,754,29,31.0,14.3,4.46
4,2023,18,18 2022,Minnesota Vikings,Minnesota Vikings 2022,0.00,0.02,0.00,21,0.02,...,2093,18,4.5,113,111,926,22,41.1,12.0,-104.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1991,25,25 1990,Green Bay Packers,Green Bay Packers 1990,0.00,0.00,0.00,0,0.00,...,2059,16,4.3,113,109,854,13,0.0,0.0,0.00
1021,1991,18,18 1990,Minnesota Vikings,Minnesota Vikings 1990,0.00,0.00,0.00,0,0.00,...,2074,12,4.1,107,100,787,14,0.0,0.0,0.00
1022,1991,31,31 1990,Phoenix Cardinals,Phoenix Cardinals 1990,0.00,0.00,0.00,0,0.00,...,2318,20,4.4,140,96,834,20,0.0,0.0,0.00
1023,1991,30,30 1990,Los Angeles Rams,Los Angeles Rams 1990,0.00,0.00,0.00,0,0.00,...,1649,17,3.9,93,109,968,18,0.0,0.0,0.00


In [65]:
co = df
print(list(co.columns))

['Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season', 'sum mvp share', 'sum dpoy share', 'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs', 'L5Y cs', 'L8Y cs', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'make playoffs', 'rk playoffs', '1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed', 'champion share', 'pso', 'team_offense_G', 'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp', 'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 'team_offense_Rushing_1stD

In [66]:
#dropping all rows that had non-playoff teams

#for x in df.index:
    #if df.loc[x,'make playoffs'] == 0:
        #df.drop(x, inplace=True)
 

## Feature Selection:

Identificaiton columns: 'Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season',
    
Prediction column:  'champion share',   

Features: (columns taken out) |: 'W', 'L', 'T', 'W-L%', 'make playoffs','rk playoffs', 'PD', 'MoV', 

Features: (filtered) |: 

In [89]:
#RandomForest Regressor
#PresentTest Approach

features = ['team_offense_PF', 'team_defense_PA','SRS', 'OSRS','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','SoS', 'DSRS',   'pso', 'team_offense_G','1 seed','2 seed','PF', 'PA','3 seed', '4 seed', '5 seed', '6 seed',
            'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

#training/testing
train = df[df["Season"] <= 2022].copy()
test = df[df["Season"] > 2022].copy()

#scaling
sc = MinMaxScaler()
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

#model parameters
rf = RandomForestRegressor(n_estimators=500, random_state=75, min_samples_split=2, max_depth=5)
rf.fit(train[features], train["champion share"])

#predicting
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Result Preview:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae1 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse1 = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse1 = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)


print("Error Metrics: ")
print("=============================================================================\n")
print("PT MAE: ", mae1)
print("PT MSE: ", mse1)
print("PT RMSE: ", rmse1)
print("-----------------------------------------------------------------------------\n")


#feature importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(20))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

print(frame.to_string())

Result Preview:

    Season                  Team  champion share  predicted champion share
0     2023    Kansas City Chiefs            1.00                  0.653377
1     2023   Philadelphia Eagles            0.75                  0.541538
2     2023   San Francisco 49ers            0.50                  0.521690
3     2023         Buffalo Bills            0.25                  0.477271
18    2023  Tampa Bay Buccaneers            0.00                  0.449902
-----------------------------------------------------------------------------

Error Metrics: 

PT MAE:  0.07267635063771316
PT MSE:  0.018946054762085718
PT RMSE:  0.13764466848405615
-----------------------------------------------------------------------------

Feature Importance: 

SRS                              0.423229
2 seed                           0.132802
1 seed                           0.132109
4 seed                           0.031506
team_offense_Passing_Yds         0.020343
3 seed                           0.01

## BackTesting:
### Part 1: Champion Average Precision
### Part 2: Top Average Precision

In [90]:
#BackTesting For Champion Accuracy

frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)

frame.sort_values(["champion share"], ascending=False).head(1) #HEAD

actual = frame.sort_values("champion share", ascending=False)
predicted = frame.sort_values("predicted champion share", ascending=False)
actual["Rank"] = list(range(1,actual.shape[0]+1))
predicted["Predicted Rank"] = list(range(1,predicted.shape[0]+1))

actual.merge(predicted, on="Team").head(1) #HEAD

#Average Precision Metric by DataQuest-----------------------------------------------------
def find_ap(frame):
    
    actual = frame.sort_values("champion share", ascending=False).head(1) #HEAD
    
    predicted = frame.sort_values("predicted champion share", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Team"] in actual["Team"].values:
            found += 1
            ps.append(found / seen)
        seen += 1
    return sum(ps)/len(ps)
find_ap(frame)
ap = find_ap(frame)

print("PT AP Champion: ", ap)

PT AP Champion:  1.0


In [91]:
#RandomForest Regressor
#PresentApproach
#BackTesting Results


#backtesting
#-------------------------------------------------------------------------------------------------------
#5 = 2006
#all data from 2001-2005 becomes training, and so on.
sc = MinMaxScaler()


Seasons = list(range(1991,2024))
aps=[]
all_predictions=[]
for season in Seasons[23:]:
        train=df[df["Season"]<season].copy()
        test=df[df["Season"]==season].copy()
        sc.fit(train[features])
        train[features]=sc.transform(train[features])
        test[features]=sc.transform(test[features])
        
        mae2 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
        mse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"])
        rmse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)
        
        rf.fit(train[features], train['champion share'])       
        predictions = rf.predict(test[features]) 
        predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
        frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)
        all_predictions.append(frame)
        aps.append(find_ap(frame))

print("Error Metrics: ")
print("=============================================================================\n")
print("PT MAE: ", mae2)
print("PT MSE: ", mse2)
print("PT RMSE: ", rmse2)
print("-----------------------------------------------------------------------------\n")

Error Metrics: 

PT MAE:  0.12053138616654904
PT MSE:  0.04779328579465953
PT RMSE:  0.21861675552129925
-----------------------------------------------------------------------------



In [92]:
print("Predicted Champion Ranks:")
aps

Predicted Champion Ranks:


[1.0,
 0.5,
 1.0,
 1.0,
 0.5,
 0.3333333333333333,
 0.25,
 0.14285714285714285,
 0.14285714285714285,
 1.0]

In [93]:
ap2 = sum(aps)/len(aps)
print("BT AP Champion:", ap2)

BT AP Champion: 0.5869047619047619


## BackTest AP Top 4

In [109]:
frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)

frame.sort_values(["champion share"], ascending=False).head(4) #HEAD

actual = frame.sort_values("champion share", ascending=False)
predicted = frame.sort_values("predicted champion share", ascending=False)
actual["Rank"] = list(range(1,actual.shape[0]+1))
predicted["Predicted Rank"] = list(range(1,predicted.shape[0]+1))

actual.merge(predicted, on="Team").head(4) #HEAD

#Average Precision Metric by DataQuest-----------------------------------------------------
def find_ap(frame):
    
    actual = frame.sort_values("champion share", ascending=False).head(4) #HEAD
    
    predicted = frame.sort_values("predicted champion share", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Team"] in actual["Team"].values:
            found += 1
            ps.append(found / seen)
        seen += 1
    return sum(ps)/len(ps)
find_ap(frame)
ap = find_ap(frame)

print("PT AP Top 4: ", ap)

PT AP Top 4:  0.875


In [110]:
#RandomForest Regressor
#PresentApproach
#BackTesting Results


#backtesting
#-------------------------------------------------------------------------------------------------------
#5 = 2006
#all data from 2001-2005 becomes training, and so on.
sc = MinMaxScaler()


Seasons = list(range(1991,2024))
aps=[]
all_predictions=[]
for season in Seasons[23:]:
        train=df[df["Season"]<season].copy()
        test=df[df["Season"]==season].copy()
        sc.fit(train[features])
        train[features]=sc.transform(train[features])
        test[features]=sc.transform(test[features])
        
        mae2 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
        mse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"])
        rmse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)
        
        rf.fit(train[features], train['champion share'])       
        predictions = rf.predict(test[features]) 
        predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
        frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)
        all_predictions.append(frame)
        aps.append(find_ap(frame))

print("Error Metrics: ")
print("=============================================================================\n")
print("PT MAE: ", mae2)
print("PT MSE: ", mse2)
print("PT RMSE: ", rmse2)
print("-----------------------------------------------------------------------------\n")

Error Metrics: 

PT MAE:  0.11926400902913911
PT MSE:  0.0480397932392705
PT RMSE:  0.21917981941609155
-----------------------------------------------------------------------------



In [111]:
print("AP Top 4 Each Season:")
aps

AP Top 4 Each Season:


[0.8541666666666666,
 0.8875,
 0.8875,
 0.8303571428571428,
 0.875,
 1.0,
 0.5625,
 0.8303571428571428,
 0.3630952380952381,
 0.875]

In [112]:
ap2 = sum(aps)/len(aps)
print("BT AP Top 4:", ap2)

BT AP Top 4: 0.796547619047619


In [94]:
#Export Prediction 
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)

frame = pd.concat([test[['Season', "Team/Season"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
frame.to_excel('NFL Champion Predict.xlsx')

#Export Importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
f_importance.to_excel('NFL Champion Fea Imp.xlsx')

In [203]:
#Seasons[23:]
#Seasons = list(range(1991,2024))
#Seasons

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]